<a href="https://www.kaggle.com/code/alibidaran/caption-recomendation-clip?scriptVersionId=136272712" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch_snippets==0.499

In [ ]:
!pip install transformers 
from transformers import AutoTokenizer,AutoModel
import torch 
from torch_snippets import *
from torchvision import transforms
from sklearn.model_selection import train_test_split
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased',use_fast=False)
model=AutoModel.from_pretrained('distilbert-base-multilingual-cased').to(device)

In [ ]:
df=pd.read_csv('/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv',delimiter="|")
df=df.dropna()

In [ ]:
df

In [ ]:
txt=df[df.columns[-1]].tolist()
images=df[df.columns[0]].tolist()


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(images,txt,test_size=0.2)

In [ ]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])


In [ ]:
class SegData(Dataset):
    def __init__(self,images,txt):
        self.image_path='/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/'
        self.images=images
        self.txt=txt
        self.tokenizer=tokenizer
    def __len__(self):
        return len(self.images)
    def __getitem__(self, ix):
        image = cv2.imread(self.image_path+self.images[ix])
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224,224))
        label=self.txt[ix]
        #tokenized=tokenizer(label,max_length=128,padding=True,truncation=True,return_tensors="pt")
        #image=tfms(image/255.)
        return image,label
    def choose(self): return self[randint(len(self.images))]
    def collate_fn(self, batch):
      ims, caps = list(zip(*batch))
      ims = torch.cat([tfms(im.copy()/255.)[None] for im in ims]).float().to(device)
      cap_= [cap for cap in caps]
      return ims, cap_

In [ ]:
train_dataset=SegData(x_train,y_train)
test_dataset=SegData(x_test,y_test)

In [ ]:
image,label=train_dataset[-1]
print(image.shape)
plt.imshow(image)
label_t=tokenizer(label,max_length=32,padding=True,truncation=True,return_tensors="pt"
                  ,add_special_tokens=True)
print(tokenizer.decode(label_t['input_ids'][0]))

In [ ]:
label_t['input_ids']

In [ ]:
inputs={i:j.to(device) for i,j in label_t.items()}

In [ ]:
with torch.no_grad():
    outputs=model(**inputs)
    print(outputs)

In [ ]:
class Text_encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.base=model
        for param in self.base.parameters():
            param.requires_grad=False
        self.l1=nn.Linear(768,512,bias=False)
        self.l2=nn.Linear(512,512,bias=False)
        self.dropout=nn.Dropout(p=0.2)
        self.gelu=nn.GELU()
        self.layernorm=nn.LayerNorm(512)
    def forward(self,x):
        output=self.base(**x)
        x1=self.l1(output.last_hidden_state[:,0])
        x2=self.dropout(self.gelu(self.l2(x1)))
        out=self.layernorm(x1+x2)
        return out/torch.norm(out,dim=-1,keepdim=True)
    

In [ ]:
text_encoder=Text_encoder().to(device)

In [ ]:
trn_dl=DataLoader(train_dataset,batch_size=128,drop_last=True,shuffle=True,collate_fn=train_dataset.collate_fn)
test_dl=DataLoader(test_dataset,batch_size=128,drop_last=True,shuffle=True,collate_fn=test_dataset.collate_fn)

In [ ]:
image,label=next(iter(trn_dl))
label_t=tokenizer(label,max_length=32,padding=True,truncation=True,return_tensors="pt",
                 add_special_tokens=True)
inputs={i:j.to(device) for i,j in label_t.items()}
with torch.no_grad():
    out=text_encoder(inputs)
    print(out)
    

In [ ]:
## adding vision encoder
from torchvision import models
backbone=models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
backbone

In [ ]:
for param in backbone.parameters():
    param.require_grad=False

In [ ]:
backbone.fc=torch.nn.Sequential(
torch.nn.Identity()
)
backbone

In [ ]:
class Image_encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.base=backbone.to(device)
        self.l1=nn.Linear(512,512,bias=False)
        self.l2=nn.Linear(512,512,bias=False)
        self.dropout=nn.Dropout(p=0.2)
        self.gelu=nn.GELU()
        self.layernorm=nn.LayerNorm(512)
    def forward(self,x):
        x1=self.l1(self.base(x))
        x2=self.dropout(self.gelu(self.l2(x1)))
        out=self.layernorm(x1+x2)
        return out/torch.norm(out,dim=-1,keepdim=True)
    

In [ ]:
image_encoder=Image_encoder().to(device)
with torch.no_grad():
    image_enc=image_encoder(image)

In [ ]:
def constraitive_loss(similarity,dim):
    prob_ls=torch.diag(F.log_softmax(similarity,dim=dim))
    return -prob_ls.mean()
def clip(sim):
    image_prob=constraitive_loss(sim,dim=0)
    text_prob=constraitive_loss(sim,dim=1)
    return (image_prob+text_prob)/2
def accuracy(sim):
    y=torch.arange(len(sim)).to(device)
    image_cap=torch.argmax(sim,dim=0)
    text_cap=torch.argmax(sim,dim=1)
    img_acc=(image_cap==y).float().mean()
    text_acc=(text_cap==y).float().mean()
    return img_acc,text_acc

In [ ]:
text_encoder=Text_encoder().to(device)
image_encoder=Image_encoder().to(device)
loss_fn=clip
params=[{'params':image_encoder.l1.parameters(),'lr':1e-3},
        {'params': image_encoder.l2.parameters(),'lr':1e-3},
        {'params':text_encoder.l1.parameters(),'lr':1e-3},
        {'params':text_encoder.l2.parameters(),'lr':1e-3},
       ]
optimizer=torch.optim.Adam(params,lr=1e-3)

In [ ]:
def train_batch(batch):
  image,label=batch
  label_t=tokenizer(label,max_length=32,padding=True,truncation=True,return_tensors="pt"
                   ,add_special_tokens=True)
  inputs={i:j.to(device) for i,j in label_t.items()}
  text_encoder.train()
  image_encoder.train()
  optimizer.zero_grad()
  image_pred=image_encoder(image)
  text_pred=text_encoder(inputs)
  sim=image_pred@text_pred.T
  loss=loss_fn(sim)
  image_acc,text_acc=accuracy(sim)
  loss.backward()
  optimizer.step()
  return loss.item(),image_acc.item(),text_acc.item()
@torch.no_grad()
def valid_batch(batch):
  image,label=batch
  label_t=tokenizer(label,max_length=32,padding=True,truncation=True,return_tensors="pt"
                   ,add_special_tokens=True)
  inputs={i:j.to(device) for i,j in label_t.items()}
  text_encoder.eval()
  image_encoder.eval()
  optimizer.zero_grad()
  image_pred=image_encoder(image)
  text_pred=text_encoder(inputs)
  sim=image_pred@text_pred.T
  loss=loss_fn(sim)
  image_acc,text_acc=accuracy(sim)
  return loss.item(),image_acc.item(),text_acc.item()

In [ ]:
n_epoch=1
log=Report(n_epoch)
for epochs in range(n_epoch):
  N = len(trn_dl)
  for i,data in enumerate(trn_dl):
    loss,image_acc,text_acc=train_batch(data)
    log.record(epochs+(i+1)/N,trn_loss=loss,trn_img_acc=image_acc,trn_text_acc=text_acc,end='\r')
  N = len(test_dl)
  for i,batch in enumerate(test_dl):
    loss,image_acc,text_acc=valid_batch(data)
    log.record(epochs+(i+1)/N,val_loss=loss,val_img_acc=image_acc,val_text_acc=text_acc,end='\r')
 

  log.report_avgs(epochs+1)

In [ ]:
import tqdm 
text_encoder.eval()
image_encoder.eval()
caps=[]
trn_dl=DataLoader(train_dataset,batch_size=32,drop_last=True,shuffle=True,collate_fn=train_dataset.collate_fn)
test_dl=DataLoader(test_dataset,batch_size=32,drop_last=True,collate_fn=test_dataset.collate_fn)
image,cap=next(iter(test_dl))

with torch.no_grad():
    image_enc=image_encoder(image)
#label_enc=text_encoder(label_t)
with torch.no_grad():
    for _,cap in tqdm.tqdm(test_dl):
        label_t=tokenizer(cap,max_length=32,padding=True,truncation=True,return_tensors="pt"
                   ,add_special_tokens=True)
        inputs={i:j.to(device) for i,j in label_t.items()}
        cap_enc=text_encoder(inputs)
        caps.append(cap_enc)


In [ ]:
text_encs=torch.cat(caps)
sim=image_enc@text_encs.T


In [ ]:
val,closest=sim.topk(5,dim=-1)
sim.size()

In [ ]:
def recommend_caption(idx):
    img,real_cap=test_dataset[idx]
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    caps=[test_dataset[closest[idx, i]][1] for i in range(5)]
    print("Recommended captions:")
    for i,cap in enumerate(caps):
        print(i,cap)
    print("Real caption:",real_cap)


In [ ]:
idxs=range(10,25,1)
for idx in idxs:
    recommend_caption(idx)